# Import

In [ ]:
import tensorflow as tf
from tensorflow import keras
!pip install -q -U keras-tuner
import keras_tuner as kt

     |████████████████████████████████| 98 kB 3.7 MB/s 


# Loading Data

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()
# Normalize pixel values between (img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()
# Normalize pixel values between 0 and 1
# img_train = img_train.astype('float32') / 255.0
# img_test = img_test.astype('float32') / 255.00 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

# Base Model

In [ ]:
def model_builder():
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  model.add(keras.layers.Dense(units=512, activation='relu'))
  model.add(keras.layers.Dense(10))
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [ ]:
model = model_builder()
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)


Epoch 1/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.5757 - accuracy: 0.8018 - val_loss: 0.4972 - val_accuracy: 0.8244
Epoch 2/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.4489 - accuracy: 0.8410 - val_loss: 0.4291 - val_accuracy: 0.8552
Epoch 3/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4139 - accuracy: 0.8515 - val_loss: 0.4100 - val_accuracy: 0.8563
Epoch 4/50
1500/1500 [==============================] - 7s 4ms/step - loss: 0.4204 - accuracy: 0.8500 - val_loss: 0.4940 - val_accuracy: 0.8365
Epoch 5/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.4005 - accuracy: 0.8569 - val_loss: 0.4443 - val_accuracy: 0.8446
Epoch 6/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3888 - accuracy: 0.8602 - val_loss: 0.4200 - val_accuracy: 0.8577
Epoch 7/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3800 - accuracy: 0.8655 - val_loss: 0.4463 - val_accuracy:

# Keras Tuner

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  dense_units = hp.Int('dense_units',
                                min_value=16,
                                max_value=512,
                                step=16)
  model.add(keras.layers.Dense(units=dense_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  lr = hp.Choice("learning_rate",
		  values=[0.0001, 0.0005, 0.001, 0.005, 0.01])
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [ ]:
tuner = kt.Hyperband(
		model_builder,
    max_epochs=10,
		objective="val_accuracy",
    executions_per_trial=3,  # how many trials per variation? (same model could perform differently)
    directory='FASHION_PROJECT',
		factor=3,
		seed=42,
		project_name='KerasTuner')

In [ ]:
NUM_EPOCH = 30
tuner.search(x=img_train, 
             y=label_train,
             epochs=NUM_EPOCH,
             batch_size=32,
             validation_split=0.3)

Trial 30 Complete [00h 03m 52s]
val_accuracy: 0.8745185335477194

Best val_accuracy So Far: 0.8901296257972717
Total elapsed time: 00h 49m 28s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in FASHION_PROJECT/KerasTuner
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
dense_units: 384
learning_rate: 0.0005
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8901296257972717
Trial summary
Hyperparameters:
dense_units: 416
learning_rate: 0.0005
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: bab51aeeee7a7769610420f410bfa908
Score: 0.8837963143984476
Trial summary
Hyperparameters:
dense_units: 464
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 765cee930baeefd62017be15a8088d27
Score: 0.8832592566808065
Trial summary
Hyperparameters:
dense_units: 224
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: fe7f68e73f51070da036932dd263617d
Score: 0.8824259241422018
Trial summary
Hyperparameters:
dense_units: 192
learning_r

In [ ]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 384)               301440    
                                                                 
 dense_1 (Dense)             (None, 10)                3850      
                                                                 
Total params: 305,290
Trainable params: 305,290
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(
    img_train, 
    label_train,
    epochs=NUM_EPOCH,
    batch_size=32,
    validation_split=0.3
)

Epoch 1/30
1313/1313 [==============================] - 7s 5ms/step - loss: 0.2185 - accuracy: 0.9188 - val_loss: 0.3157 - val_accuracy: 0.8867
Epoch 2/30
1313/1313 [==============================] - 7s 5ms/step - loss: 0.2109 - accuracy: 0.9215 - val_loss: 0.3083 - val_accuracy: 0.8925
Epoch 3/30
1313/1313 [==============================] - 7s 6ms/step - loss: 0.2000 - accuracy: 0.9269 - val_loss: 0.3083 - val_accuracy: 0.8937
Epoch 4/30
1313/1313 [==============================] - 6s 5ms/step - loss: 0.1932 - accuracy: 0.9283 - val_loss: 0.3157 - val_accuracy: 0.8917
Epoch 5/30
1313/1313 [==============================] - 6s 5ms/step - loss: 0.1857 - accuracy: 0.9303 - val_loss: 0.3082 - val_accuracy: 0.8944
Epoch 6/30
1313/1313 [==============================] - 7s 5ms/step - loss: 0.1784 - accuracy: 0.9329 - val_loss: 0.3028 - val_accuracy: 0.8972
Epoch 7/30
1313/1313 [==============================] - 6s 4ms/step - loss: 0.1717 - accuracy: 0.9363 - val_loss: 0.3341 - val_accuracy: